## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_16.f'
SUBMIT = 'submit/course_16.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 181)

In [4]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 181)


,No,ball,course,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course00,bc_course01,bc_course02,bc_course03,bc_course04,bc_course05,bc_course06,bc_course07,bc_course08,bc_course09,bc_course10,bc_course11,bc_course12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,...,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,0,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,0,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,0,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,1,1,0,0,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,1,1,0,0,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [5]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 181)

### test
- 行数: 521650

In [6]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 181)

In [7]:
del all_pitch
gc.collect()

60

In [8]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [9]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [10]:
lgb_train = lgb.Dataset(train_d, train['course'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [11]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 8.769293390201968, 
        'lambda_l2': 3.913949617576324e-05, 
        'num_leaves': 6, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.8391111798378441, 
        'bagging_freq': 4, 
        'min_child_samples': 50
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.32538 + 0.000729413
[200]	cv_agg's multi_logloss: 2.32114 + 0.00078532
[300]	cv_agg's multi_logloss: 2.32039 + 0.000921575
[400]	cv_agg's multi_logloss: 2.32037 + 0.000983582
Best num_boost_round: 355
Best CV score: 2.3203553166835866
CPU times: user 40min 47s, sys: 8.3 s, total: 40min 55s
Wall time: 5min 8s


In [12]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [13]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 11min 5s, sys: 2.07 s, total: 11min 7s
Wall time: 1min 23s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
8,pitcher_cnt_in_inning,274
47,elapsed_min,276
22,bc_fork,286
7,pitcher_cnt_in_game,292
169,salary_year_dif_b-c,300
103,course_5,306
102,course_4,323
115,height_bat,327
157,bmi_dif_p-b,330
101,course_3,332


In [15]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 2min 30s, sys: 756 ms, total: 2min 31s
Wall time: 20 s


## 結果出力

In [16]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.058623,0.131925,0.065878,0.037556,0.065146,0.030982,0.055265,0.062547,0.039503,0.167777,0.090125,0.101743,0.092930
1,1,0.053783,0.141413,0.068840,0.031136,0.060545,0.032488,0.048619,0.068480,0.035956,0.150735,0.081856,0.116585,0.109566
2,2,0.034906,0.086898,0.053487,0.024440,0.040331,0.023431,0.044459,0.060658,0.028220,0.156206,0.105148,0.205024,0.136793
3,3,0.044954,0.107299,0.062823,0.026353,0.043838,0.024614,0.044677,0.059516,0.035331,0.170849,0.108832,0.146827,0.124086
4,4,0.056951,0.130496,0.066417,0.035900,0.046994,0.030556,0.048047,0.069077,0.044561,0.140288,0.080713,0.128541,0.121461


In [17]:
submit.mean()

index    260824.500000
0             0.046847
1             0.068777
2             0.056574
3             0.046458
4             0.059745
5             0.046970
6             0.043153
7             0.064068
8             0.052963
9             0.110210
10            0.088443
11            0.137125
12            0.178668
dtype: float64

In [18]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [19]:
'signate submit --competition-id=276 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ./submit/course_16.csv --note feat=181_cv=2.3203553166835866'

### GCS

In [20]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/course_16.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 16(181)  ...選手IDごとの2017年のコース実績
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 355
- Best CV score: 2.3203553166835866
- Wall time: 5min 8s
- 評価結果  : 2.3608998

#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648